# Trabalho Prático 1

## Introdução

A resolução deste trabalho prático tem como objetivo aprofundar o conhecimento na implementação de sessões cifradas e seguras num ambiente entre um emissor e um recetor.
Os objetivos principais deste trabalho prático são:

- A construção de uma sessão de comunicação síncrona entre um emissor e um recetor.
- A utilização do protocolo de **Diffie_Hellman** com verificação de chave e autenticação dos agentes na sessão anteriormente referida.
- Utilização do **DSA** para a autenticação dos agentes.
- Utilização do **TAES** como cifra simétrica, com autenticação do criptograma em cada superbloco.
- Finalmente, o último objetivo passa por implementar novamente o cenário anterior mas, desta feita, utilizando o **ECDH** (Elliptic Curve Diffie-Hellman) em vez do **DH** e o **ECDSA** (Elliptic Curve Digital Signature Algorithm) em vez do **DSA**.

Este relatório está escrito de forma a que o texto escrito entre o código desenvolvido seja suficientemente explicativo do que está a ser implementado, o que, na nossa opinião, permite uma leitura e compreensão facilitada do mesmo.

## Sessão Síncrona com _Diffie-Hellman_

### Imports

Esta secção tem como objetivo importar os módulos **Python** necessários para desenvolver a sessão síncrona entre os agentes, utilizando o protocolo **DH** e o **DSA**.

In [1]:
import os
from multiprocessing import Process,Pipe
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

### Definição da classe de multiprocessamento

Esta secção tem o propósito de definir a classe de multiprocessamento, que permite uma comunicação bidireccional com o _Emitter_ e o _Receiver_, sendo estes dois processos criados e implementados pela **API** [**multiprocessing**](https://docs.python.org/2/library/multiprocessing.html).

In [5]:
class BiConnection(object):
    def __init__(self,left,right):
        left_side,right_side = Pipe()
        self.timeout = None
        self.left_process = Process(target=left,args=(left_side,))
        self.right_process = Process(target=right,args=(right_side,))
        self.left = lambda : left(left_side)
        self.right = lambda : right(right_side)
    
    def manual(self): # Execução manual apenas devido ao facto de a password ter que ser lida em ambos os lados do Pipe
        self.left()
        self.right()

### Geração dos parâmetros _Diffie-Hellman_

Esta secção tem o objetivo de gerar os parâmetros necessários para a geração de chaves por parte de ambos os agentes.

In [ ]:
parameters = dh.generate_parameters(generator=2,key_size=2048,backend=default_backend())

### Definição do Emissor e do Recetor

Nesta secção está definido o comportamento de cada um dos agentes ao utilizar esta sessão síncrona segura de troca de informação.

In [3]:
def Emissor(connection):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-3-60a3f5d4e584>, line 2)

In [4]:
def Recetor(connection):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-4-8bc75b87e778>, line 2)